# Create DA Lookup Table

This notebook generates a lookup table between Census Dissemination Area and municipality.

In [9]:
import pandas as pd
from google.cloud import bigquery
from urllib.request import urlretrieve

In [3]:
urlretrieve(url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/dguid-idugd/files-fichiers/2021_98260004.zip", filename = "relationship_file.zip")
!unzip relationship_file.zip

Archive:  relationship_file.zip
  inflating: 2021_98260004.csv       


In [10]:
relationship_data = pd.read_csv("./2021_98260004.csv", dtype = str)
relationship_data.head()

,PRDGUID_PRIDUGD,CDDGUID_DRIDUGD,FEDDGUID_CEFIDUGD,CSDDGUID_SDRIDUGD,ERDGUID_REIDUGD,CARDGUID_RARIDUGD,CCSDGUID_SRUIDUGD,DADGUID_ADIDUGD,DBDGUID_IDIDUGD,ADADGUID_ADAIDUGD,DPLDGUID_LDIDUGD,CMAPDGUID_RMRPIDUGD,CMADGUID_RMRIDUGD,CTDGUID_SRIDUGD,POPCTRPDGUID_CTRPOPPIDUGD,POPCTRDGUID_CTRPOPIDUGD
0,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165001,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
1,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165002,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
2,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165006,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
3,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165007,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
4,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010166,2021S051310010166001,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792


In [13]:
client = bigquery.Client()
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    relationship_data, 
    'row-community-database.unprocessed.census_relationship', 
    job_config=job_config
)  
job.result()

LoadJob<project=row-community-database, location=northamerica-northeast2, id=b6ccfe8a-8a11-4a49-9230-debfef8c78d6>

In [ ]:
!rm relationship_file.zip
!rm 2021_98260004.csv